# Importing libraries

In [ ]:
!pip install yfinance pandas numpy matplotlib seaborn scikit-learn keras

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# Introduction

## Goal
Use a Long Short-Term Memory (LSTM) model to predict the stock price of Ford Motor Company.

## Why LSTM?
LSTM is a type of Recurrent Neural Network (RNN) built specifically to learn patterns in time-series data — especially when past information matters, as it does for predicting stock prices based on historical data.


# Get Ford's Stock Data
In order to get the stock data for the Ford stock we'll use **Yahoo Finance** (yfinance)

In [ ]:
stock = 'F'
data = yf.download(stock, start = '2016-01-01', end = '2025-01-01')
data.head()

With the data now obtained we can visualize it to better understand the data.

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(data['Close'])
plt.title(f'{stock} Closing Price')
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

# Preprocess Data for the LSTM

In [ ]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data['Close'])

def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

sequence_length = 60
X, y = create_sequences(scaled_data, sequence_length)

X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Build the LSTM Model

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train/Test split

In [ ]:
training_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:training_size]
test_data = scaled_data[training_size - 60:]

X_train, y_train = create_sequences(train_data, sequence_length)
X_test, y_test = create_sequences(test_data, sequence_length)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Train the model

In [ ]:
model.fit(X, y, epochs=20, batch_size=32)

# Predict and Visualize

In [ ]:
predicted = model.predict(X_test)
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1,1))

plt.figure(figsize=(10,6))
plt.plot(real_prices, label='Actual')
plt.plot(predicted_prices, label='Predicted')
plt.legend()
plt.title('Actual vs Predicted Closing Price')
plt.show()